In [1]:
!pip install simpleT5 datasets

  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.1
    Uninstalling transformers-4.40.1:
      Successfully uninstalled transformers-4.40.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
arviz 0.15.1 requires setuptools>=60.0.0, but you have setuptools 59.5.0 which is incompatible.
cvxpy 1.3.4 requires setuptools>65.5.1, but you have setuptools 59.5.0 which is incompatible.


In [2]:
from simplet5 import SimpleT5

model = SimpleT5()

model.from_pretrained("t5","t5-small")

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Downloading:   0%|          | 0.00/2.27k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/231M [00:00<?, ?B/s]

In [3]:
import pandas as pd

from datasets import load_dataset

sst2_df =pd.DataFrame(

    load_dataset("glue",

                 "sst2",

                  split="train[:1000]"))

sst2_df=sst2_df[["sentence","label"]]

sst2_df.columns= ["source_text","target_text"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [6]:
prompt_sentiment=". What is the sentiment ? " + "Good or Bad ?"

sst2_df["source_text"]= sst2_df.source_text.apply(lambda x: x +prompt_sentiment)

sst2_df["target_text"]=sst2_df.target_text.apply(lambda x:"Good" if x==1 else "Bad")

In [7]:
opus=load_dataset("opus100","de-en",

                  split="train[:1000]")

opus_df=pd.DataFrame(opus)

opus_df["source_text"]=opus_df.apply(

    lambda x: x.translation["en"], axis=1)

opus_df["target_text"]=opus_df.apply(

    lambda x: x.translation["de"], axis=1)

opus_df= opus_df[["source_text","target_text"]]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [8]:
prompt_tr=". Translate English to German"

opus_df["source_text"]= opus_df.source_text.apply(lambda x: x +prompt_tr)

In [10]:
merge= opus_df._append(sst2_df).sample(frac=1.0)

merge.head(5)

,source_text,target_text
860,a perfect performance . What is the sentiment ...,Good
353,Aw.. Translate English to German,- Was Neues von Tara?
333,a clunky tv-movie approach to detailing a chap...,Bad
905,So what do you want from me?. Translate Englis...,"- Nun, das freut mich für sie."
289,warm water under a red bridge is a celebration...,Good


In [11]:
train_df=merge[:1800]

eval_df= merge[1800:]

In [ ]:
model.train(train_df=train_df,

            eval_df=eval_df,

            source_max_token_len = 512,

            target_max_token_len = 128,

            batch_size = 8,

            max_epochs = 3,

            use_gpu = True,

            precision = 32)

In [ ]:
import torch

model.device= torch.device("cpu")

a_sentence="The cats are fun!"

model.predict(a_sentence+ prompt_sentiment)